In [ ]:
!pip install --upgrade gokart thunderbolt --quiet

     |████████████████████████████████| 132 kB 29.4 MB/s 
     |████████████████████████████████| 244 kB 66.0 MB/s 
     |████████████████████████████████| 265 kB 68.3 MB/s 
     |████████████████████████████████| 59 kB 3.1 MB/s 
     |████████████████████████████████| 1.2 MB 33.4 MB/s 
     |████████████████████████████████| 70 kB 4.4 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 8.9 MB 46.7 MB/s 
     |████████████████████████████████| 138 kB 11.7 MB/s 
     |████████████████████████████████| 127 kB 42.9 MB/s 
     |████████████████████████████████| 339 kB 10.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [15]:
import datetime
import pandas as pd
import gokart

from sklearn.datasets import load_iris


tmp_dir_path = './tmp/'
output_dir_path = './output/'

class GetData(gokart.TaskOnKart):
    task_namespace = 'tutorial'

    def output(self):
        return self.make_target(tmp_dir_path + 'GetData.csv')

    def run(self):
        # get data
        input_df, _ = load_iris(return_X_y=True, as_frame=True)
        # save input data with tie file path './tmp/GetData_{unique_id}.csv
        self.dump(input_df)


class PreprocessA(gokart.TaskOnKart):
    task_namespace = 'tutorial'

    def requires(self):
        return GetData()

    def output(self):
        return self.make_target(tmp_dir_path + 'PreprocessA.csv')

    def run(self):
        # load data which GetData output
        input_df = self.load()
        # preprocess with input_df, and make result_df
        result_df = input_df['sepal length (cm)']
        # save results with the file path '.tmp/PreprocessA_{unique_id}.csv'
        self.dump(result_df)


class PreprocessB(gokart.TaskOnKart):
    task_namespace = 'tutorial'

    def requires(self):
        return GetData()

    def output(self):
        return self.make_target(tmp_dir_path + 'PreprocessB.csv')

    def run(self):
        # load data which GetData output
        input_df = self.load('a')
        # preprocess with input_df, and make result_df.
        result_df = input_df['sepal width (cm)']
        # save results with the file path '.tmp/PreprocessB_{unique_id}.csv'
        self.dump(result_df)

class Sample(gokart.TaskOnKart):
    task_namespace = 'tutorial'

    def requires(self):
        return dict(a=PreprocessA(), b=PreprocessB())

    def output(self):
        # save results with the file path './output/result_{unique_id}.csv'
        return self.make_target(output_dir_path + 'result.csv')
 
    def run(self):
        # input a
        input_a = self.load('a')
        # input b
        input_b = self.load('b')
        # process (marge)
        df = pd.concat([input_a, input_b], axis=1)
        # output
        self.dump(df)

In [16]:
task = Sample()
output = gokart.build(task)
print(output)

     sepal length (cm)  sepal width (cm)
0                  5.1               3.5
1                  4.9               3.0
2                  4.7               3.2
3                  4.6               3.1
4                  5.0               3.6
..                 ...               ...
145                6.7               3.0
146                6.3               2.5
147                6.5               3.0
148                6.2               3.4
149                5.9               3.0

[150 rows x 2 columns]


In [ ]:
from thunderbolt import Thunderbolt

tb = Thunderbolt(workspace_directory='resources', use_tqdm=True)
display(tb.get_task_df())

100%|██████████| 4/4 [00:00<00:00, 6262.49it/s]


,task_id,task_name,last_modified,task_params
0,0,GetData,2022-06-05 10:05:53.802990,{}
1,1,PreprocessB,2022-06-05 10:05:53.810990,{}
2,2,PreprocessA,2022-06-05 10:05:53.817991,{}
3,3,Sample,2022-06-05 10:05:53.828991,{}


In [ ]:
display(tb.get_data('GetData'))

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [19]:
from abc import ABCMeta, abstractmethod
from dataclasses import dataclass

@dataclass(frozen=True)
class Pipeline(metaclass=ABCMeta):
    @abstractmethod
    def require(self):
        pass

    def load(self, target: str=None):
        if target:
            return self.require()[target]
        return self.require()
    
    @abstractmethod
    def run(self):
        pass

    def dump(self, obj, target: str=None):
        

NameError: ignored

In [11]:
class PipelineA(Pipeline):
    def export(self):
        return 1

class PipelineB(Pipeline):
    def require(self):
        return PipelineA()

    def load(self):
        output_A = self.require().export()
        return output_A + 1

In [12]:
PipelineB().load()

2